## Keara Hayes, Atticus Chong, Zachary Malzahn, Evelyn Fuhrman

In [1]:
import numpy as np
from numpy.linalg import norm
import math

1) From ODE.py

In [ ]:
alpha = 4*np.pi**2

In [2]:
def fEuler(f,t,z,h,args=()):
    """    
    Arguments
        f(t,z,...)
            function that contains the RHS of the equation dz/dt = f(t,z,...)
    
        t (scalar)
            current time
            
        z (array-like)
            function value
            
        h (scalar)
            step size
    
        args (tuple, optional)
            additional arguments to pass to f
    
    Returns
        znew = z(t+h)
    """
    
    # The following trick allows us to pass additional parameters to f
    # first we make sure that args is of type tuple; if not, we make it into
    # that form
    if not isinstance(args,tuple):
        args = (args,)
    
    # when we call f, we use *args to pass it as a list of parameters.
    # for example, if elsewhere we define f like
    # def f(t,z,x,y):
    #    ...
    # then we would call this routine as
    # znew = fEuler(f,t,z,h,args=(x,y))
    #
    return z + h*f(t,z,*args)

In [3]:
def rk2(f,t,z,h,args=()):
    """
    <Description of routine goes here: what it does, how it's called>
    
    Arguments
        f(t,z,...)
            function that contains the RHS of the equation dz/dt = f(t,z,...)
    
        t (scalar)
            final time (which we're iterating to)
            
        z (array-like)
            function value
            
        h (scalar)
            step size
    
        args (tuple, optional)
            additional arguments to pass to f
    
    Returns
        znew = z(t+h)
    """

    zp = z + (h/2)*f(t,z,*args)
    
    if not isinstance(args,tuple):
        args = (args,)
    
    return z + h*f(t + h/2, zp, *args)

In [4]:
def rk4(f,t,z,h,args=()):
    """
    <Description of routine goes here: what it does, how it's called>
    
    Arguments
        f(t,z,...)
            function that contains the RHS of the equation dz/dt = f(t,z,...)
    
        t (scalar)
            final time (which we're iterating to)
            
        z (array-like)
            function value
            
        h (scalar)
            step size
    
        args (tuple, optional)
            additional arguments to pass to f
    
    Returns
        znew = z(t+h)
    """
   
    if not isinstance(args,tuple):
        args = (args,)

    k1 = f(t, z, *args)

    k2 = f(t + h/2, z + (h/2)*k1, *args)

    k3 = f(t + h/2, z + (h/2)*k2, *args)

    k4 = f(t + h, z + h*k3, *args)
    
    return z + (h/6)*(k1 + 2*k2 + 2*k3 + k4)

2) from kepler.py

In [5]:
# use this to identify methods
integration_methods = {
    'Euler':fEuler,
    'RK2':rk2,
    'RK4':rk4
    }

In [6]:
# energies
def kinetic_energy(v):
    """
    Returns kinetic energy per unit mass: KE(v) = 0.5 v*v.
    
    Arguments
        v (array-like)
            velocity vector
    """
    return 0.5*np.linalg.norm(v)**2

def potential_energy(x,m):
    """
    Returns potential energy per unit mass: PE(x, m) = -m/norm(r)

    Arguments
        x (array-like)
            position vector
        m (scalar)
            total mass in normalized units
    """
    # the norm function returns the length of the vector
#     r = norm(x)
    return -alpha*m/np.linalg.norm(r)

def total_energy(x,z,m):
    """
    Returns energy per unit mass: E(z,m) = KE(v) + PE(x,m)

    Arguments
        x (array-like)
            position array
            
        z (array-like)
            function value
            
        m (scalar)
            total mass in normalized units
    """
    # to break z into position, velocity vectors, we use array slices:
    # here z[n:m] means take elements of z with indices n <= j < m
    r = z[0:2]  # start with index 0 and take two indices: 0 and 1
    v = z[2:4]  # start with index 2 and take two indices: 2 and 3

    # replace the following two lines
    return kinetic_energy(v) + potential_energy(x,m)

In [7]:
def derivs(t,z,m):
    """
    Computes derivatives of position and velocity for Kepler's problem 
    
    Arguments
        t (scalar)
            final time (which we're iterating to)
            
        z (array-like)
            function value at supplied t, z, etc
            
        m (scalar)
            total mass in normalized units
            
    Returns
        numpy array dzdt with components [ dx/dt, dy/dt, dv_x/dt, dv_y/dt ]
    """

    r = z[0:2]
    v = z[2:4]
    rabs = np.sqrt(np.sum(r**2))

    drdt = v
    
    # scaled to the solar system
    dvdt = -4*(np.pi**2)*r/(rabs**3)
    
    # join the arrays
    dzdt = np.concatenate((drdt,dvdt))
    return dzdt

3)

In [8]:
def integrate_orbit(z0,m,tend,h,method='RK4'):
    """
    Integrates orbit starting from an initial position and velocity from t = 0 
    to t = tend.
    
    Arguments:
        z0 (scalar)
            initial function value
        m (scalar)
            mass of star
    
        tend (scalar)
            final timestep
    
        h (scalar)
            incrementing stepsize
    
        method ('Euler', 'RK2', or 'RK4')
            identifies which stepper routine to use (default: 'RK4')
    Returns
        ts, Xs, Ys, KEs, PEs, TEs := arrays of time, x postions, y positions, 
        and energies (kin., pot., total) 
    """

    # set the initial time and phase space array
    t = 0.0
    z = z0

    # expected number of steps
    Nsteps = int(tend/h)+1

    # arrays holding t, x, y, kinetic energy, potential energy, and total energy
    ts = np.zeros(Nsteps) # time
    Xs = np.zeros(Nsteps) # x-axis position
    Ys = np.zeros(Nsteps) # y-axis position
    KEs = np.zeros(Nsteps) # kinetic energy
    PEs = np.zeros(Nsteps) # potential energy
    TEs = np.zeros(Nsteps) # total energy

    # store the initial point
    ts[0] = t
    Xs[0] = z[0]
    Ys[0] = z[1]
    # now extend this with KEs[0], PEs[0], TEs[0]

    # select the stepping method
    advance_one_step = integration_methods[method]
    # run through the steps
    for step in range(1,Nsteps):
        z = advance_one_step(derivs,t,z,h,args=m)
        t += h
        # store values
        ts[step] = t
        v[step] = derivs(t,z,m) # define velocity at this step
        acc[step] = derivs(t,z,m) # define acceleration at this step
        # fill in with assignments for Xs, Ys, KEs, PEs, TEs
        Xs[step] = z[0][step] #come back to these two if you have issues...
        Ys[step] = z[1][step]
        rsquared = (Xs[step]**2) + (Ys[step]**2)
        r = np.sqrt(rsquared) 
        KEs[step] = kinetic_energy(v)
        PEs[step] = potential_energy(x,m)
        TEs[step] = KEs[step] + PEs[step]
    return ts, Xs, Ys, KEs, PEs, TEs
    

def set_initial_conditions(a,m,e):
    eps0=((a/m)*2)**-1
    T=(np.pi/math.sqrt(2))*m*(eps0**(-3/2))
    x0=(1+e)*a
    y0=0
    vx0=0
    vy0=math.sqrt(2*eps0+2*m/x0)
    r_init=np.array([x0,y0])
    v_init=np.array([vx0,vy0])
    return r_init,v_init,eps0,T

4) $$\hphantom{0}$$

In [13]:
t=2*np.pi
h_0=0.1*t
h_arr=h_0/(2**np.arange(0,11))
def dv(t,z,m):
    return (-m/(np.linalg.norm(z)**3))*z*t
def dz(t,z,m,v):
    return v+dvdt(z,m)*t
def integrate(zfunc,vfunc,method,tf,dt,m):
    pv_init,eps0=set_initial_conditions(1,1,0.5)
    x_old,y_old,vx_old,vy_old=pv_init
    t_arr=np.array([0])
    z_arr=np.array([[x_old,y_old]])
    v_arr=np.array([[vx_old,vy_old]])
    for tcurr in np.arange(0,tf,dt):
        z_arr=np.append(z_arr,method(zfunc,t_arr[-1]+dt,z_arr[-1],h,args=[m,v_arr[-1]]))
        v_arr=np.append(v_arr,method(vfunc,t_arr[-1]+dt,v_arr[-1],h,args=[m]))
        t_arr=np.append(t_arr,tcurr+dt)
    return t_arr, z_arr, v_arr

In [ ]:
v0x = 0
v0y = 2*np.pi

z0 = np.array([0,1,0,2*np.pi])

pv_init,eps0=set_initial_conditions(1,1,0.5)
t=2*np.pi
h_0=0.1*t
h_arr=h_0/np.arange(0,11)
for h in h_arr:
    integrate_orbit(z0, 1, tend = 20, h = 0.1, method = 'Euler')
    
#     (z0,m,tend,h,method='RK4')

In [9]:
pv_init,eps0=set_initial_conditions(1,1,0.5)
def dzdt(t,z,m,v):
    dvdt=(-m/(np.linalg.norm(z)**3))*z
    return v+dvdt*t
fEuler(dzdt,3,pv_init,0.1,args=(1,pv_init[2:]))

NameError: name 'G' is not defined